# DECON-EQTL PART2 : Running Decon-eQTL
### (in biowulf)

### Nov 13, 2020

In [7]:
import pandas as pd


cohort = 'nabec'
date = 'nov2020'
CHROMOSOMES=[str(x) for x in list(range(1,23))]
#WRKDIR = f'/labshare/anni/eqtl/Decon-eQTL/{cohort}'
WRKDIR = f'/data/LNG/anni/decon-eqtl/{cohort}'

#expr_in = 
geno_dir = f'{WRKDIR}/genotypes'
expr_dir = f'{WRKDIR}/expression'
fract_in = f'{WRKDIR}'


fract_file = 'nabec.scaden.v4_200.cell_fract.subset.txt'
subset_samples_file = 'nabec.deconeqtl.subset.samples.txt'
gencode_annots_file = f'{WRKDIR}/references/gencode.v32.transcripts.txt'
genome_index_file = f'{WRKDIR}/references/Homo_sapiens_assembly38.fasta.fai'



## generating sn file

### setting up environment

In [88]:
#read in the transcript annotations from txt
#gencode_annots_file = WRKDIR + '/gencode.v32.transcripts.txt.gz'

gencode = pd.read_csv(gencode_annots_file,sep='\t',comment='#', index_col=0)
print(gencode.shape)
gencode.head()

(227462, 25)


,seqname,source,feature,start,end,score,strand,frame,gene_id,gene_type,...,transcript_type,transcript_name,transcript_support_level,tag,havana_transcript,exon_number,exon_id,ont,protein_id,ccdsid
1,chr1,HAVANA,transcript,11869,14409,NaN,+,0,ENSG00000223972.5,transcribed_unprocessed_pseudogene,...,lncRNA,DDX11L1-202,1.0,basic,OTTHUMT00000362751.1,NaN,NaN,NaN,NaN,NaN
5,chr1,HAVANA,transcript,12010,13670,NaN,+,0,ENSG00000223972.5,transcribed_unprocessed_pseudogene,...,transcribed_unprocessed_pseudogene,DDX11L1-201,NaN,basic,OTTHUMT00000002844.2,NaN,NaN,"PGO:0000005,PGO:0000019",NaN,NaN
13,chr1,HAVANA,transcript,14404,29570,NaN,-,0,ENSG00000227232.5,unprocessed_pseudogene,...,unprocessed_pseudogene,WASH7P-201,NaN,basic,OTTHUMT00000002839.1,NaN,NaN,PGO:0000005,NaN,NaN
26,chr1,ENSEMBL,transcript,17369,17436,NaN,-,0,ENSG00000278267.1,miRNA,...,miRNA,MIR6859-1-201,NaN,basic,NaN,NaN,NaN,NaN,NaN,NaN
29,chr1,HAVANA,transcript,29554,31097,NaN,+,0,ENSG00000243485.5,lncRNA,...,lncRNA,MIR1302-2HG-202,5.0,"not_best_in_genome_evidence,dotter_confirmed,b...",OTTHUMT00000002840.1,NaN,NaN,NaN,NaN,NaN


### prep scripts to generate gene/snp list per chrom

In [67]:
print('run in terminal: module load plink/2.0-dev-20190318')

run in terminal: module load plink/2.0-dev-20190318


In [94]:
##section modified from raph's ppmi_wd_eqtl.ipynb script

#run plink to get all SNPs for each gene determined to be on chrom based on start/end sites
#combine all gene/SNP output file to one list
!mkdir /data/LNG/anni/decon-eqtl/nabec/plink
!mkdir /data/LNG/anni/decon-eqtl/nabec/plink/output
!mkdir /data/LNG/anni/decon-eqtl/nabec/plink/scripts
!mkdir /data/LNG/anni/decon-eqtl/nabec/plink/output/gene_snps
!mkdir /data/LNG/anni/decon-eqtl/nabec/plink/output/temp


#chrom=21
total_transcripts = 0
for chrom in CHROMOSOMES:

    exp_df = pd.read_csv(f'{WRKDIR}/expression/nabec.aug2020.chr{chrom}.subset.pheno', sep='\t', index_col=0)


    #genome_index_file = f'{WRKDIR}/Homo_sapiens_assembly38.fasta.fai'
    chr_lengths_df = pd.read_csv(genome_index_file, header=None, sep='\t')

    max_dist = 1000000

    #for chrom in CHROMOSOMES:
    #chr_genes = gencode.loc[gencode['seqid'] == 'chr{}'.format(chrom)]
    chr_genes = gencode.loc[gencode['seqname'] == 'chr{}'.format(chrom)]

    chr_genes_id = set(chr_genes['gene_id']) & set(exp_df.index)
    print('{} transcripts detected on chromosome {}'.format(len(chr_genes_id),chrom))
    total_transcripts += int(len(chr_genes_id))
    chr_genes_df = chr_genes.loc[chr_genes['gene_id'].isin(chr_genes_id)]

    #compute the +/- cis proximal start and stop
    chr_genes_df['cisstart'] = chr_genes_df['start'] - max_dist
    chr_genes_df['cisstop'] = chr_genes_df['end'] + max_dist    

    #clean up if cis start or end is beyond chromosome beginning or end
    chr_genes_df.loc[chr_genes_df['cisstart'] < 0, 'cisstart'] = 0
    max_end = chr_lengths_df.loc[chr_lengths_df[0] == 'chr{}'.format(chrom),1].values[0]
    chr_genes_df.loc[chr_genes_df['cisstop'] > max_end, 'cisstop'] = max_end

    chr_genes_df = chr_genes_df[['gene_id', 'seqname', 'start', 'end', 'strand', \
                                            'cisstart', 'cisstop', 'gene_name']]

    # chr_detected_out_file = '{}/plink/output/snp/detected.genes.chr{}'.format(WRKDIR,chrom)
    # chr_genes_df.to_csv(chr_detected_out_file, sep='\t', index=False, header=True)

    #now format the biowulf job files
    command_plink2 = 'plink --silent --pfile {A_WRKDIR}/genotypes/nabec.aug2020.chr{A_CHROM}.geno05maf05hwe000001 \
    --chr chr{A_CHROM} --from-bp {A_FROMBP} --to-bp {A_TOBP} \
    --pheno-name {A_PHENONAME} \
    --keep {A_WRKDIR}/{A_SUBFILE} \
    --make-pgen \
    --out {A_WRKDIR}/plink/output/temp/{A_PHENONAME}\n'


    OUTHEADER='\"GENEID\\tSNPID\"'
    #     cmd_accumulate = 'awk -v OFS="'"\\t"'" '"'/^#CHROM/{{ r=NR }}r && NR>r {{ print "'"{A_PHENONAME}"'","'"chr"'"$1"'":"'"$2"'":"'"$4"'":"'"$5 }}'"' \
    #     {A_WRKDIR}/plink/output/temp/{A_VISIT}/chr{A_CHROM}/{A_PHENONAME}.pvar >> {A_WRKDIR}/plink/output/snp/{A_VISIT}.genes.chr{A_CHROM}.plink.snp.list\n'
    cmd_echo_header = 'echo -e {A_HEADER} > {A_WRKDIR}/plink/output/gene_snps/chr{A_CHROM}.{A_PHENONAME}.plink.snp.list\n'
    cmd_format = 'awk -v OFS="'"\\t"'" '"'/^#CHROM/{{ r=NR }}r && NR>r {{ print "'"{A_PHENONAME}"'","'"chr"'"$1"'":"'"$2"'":"'"$4"'":"'"$5 }}'"' \
    {A_WRKDIR}/plink/output/temp/{A_PHENONAME}.pvar >> {A_WRKDIR}/plink/output/gene_snps/chr{A_CHROM}.{A_PHENONAME}.plink.snp.list\n'

    cmd_rm_result = 'rm {A_WRKDIR}/plink/output/temp/{A_PHENONAME}*\n'




    #cmd_gzip = 'gzip {A_WRKDIR}/plink/output/snp/{A_VISIT}.genes.chr{A_CHROM}.plink.snp.list\n'


    #generate chr job file for each visit
    #for visit in visits_oi:
    chr_job_file_name = f'{WRKDIR}/plink/scripts/nabec.snp.list.chr{chrom}.jobs'       
    with open(chr_job_file_name,'w') as chr_job_file:                                    
        for gene_id in chr_genes_id:
            gene_transcripts_df =  chr_genes_df.loc[chr_genes_df['gene_id'] == gene_id]
            cis_start = min(gene_transcripts_df['cisstart'])
            cos_stop = max(gene_transcripts_df['cisstop'])    
            chr_job_file.write(command_plink2.format(A_WRKDIR=WRKDIR, \
                                                     A_CHROM=chrom, \
                                                     A_PHENONAME=gene_id, A_FROMBP=cis_start, \
                                                     A_TOBP=cos_stop, A_SUBFILE = subset_samples_file))
            chr_job_file.write(cmd_echo_header.format(A_HEADER=OUTHEADER, \
                                                      A_WRKDIR=WRKDIR, \
                                                      A_PHENONAME=gene_id, A_CHROM=chrom)) 
            chr_job_file.write(cmd_format.format(A_PHENONAME=gene_id, A_WRKDIR=WRKDIR, \
                                                     A_CHROM=chrom))
            chr_job_file.write(cmd_rm_result.format(A_WRKDIR=WRKDIR, \
                                                    A_PHENONAME=gene_id))        
        #now output cmds for zipping up chr files
        #chr_job_file.write(cmd_gzip.format(A_WRKDIR=WRKDIR, A_VISIT=visit, A_CHROM=chrom))
        
print(f'total genes: '+str(total_transcripts))

1675 transcripts detected on chromosome 1


/home/mooreank/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/mooreank/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1185 transcripts detected on chromosome 2
1024 transcripts detected on chromosome 3
671 transcripts detected on chromosome 4
853 transcripts detected on chromosome 5
880 transcripts detected on chromosome 6
853 transcripts detected on chromosome 7
647 transcripts detected on chromosome 8
700 transcripts detected on chromosome 9
646 transcripts detected on chromosome 10
898 transcripts detected on chromosome 11
951 transcripts detected on chromosome 12
334 transcripts detected on chromosome 13
570 transcripts detected on chromosome 14
610 transcripts detected on chromosome 15
777 transcripts detected on chromosome 16
1001 transcripts detected on chromosome 17
288 transcripts detected on chromosome 18
1114 transcripts detected on chromosome 19
415 transcripts detected on chromosome 20
194 transcripts detected on chromosome 21
402 transcripts detected on chromosome 22
total genes: 16688


In [96]:
#format the swarm jobfile(s)
#for visit in visits_oi:
swarm_file_name = f'{WRKDIR}/plink/scripts/run_nabec_gene_snps.swarm'

with open(swarm_file_name,'w') as swarm_file:
    for chrom in CHROMOSOMES:
        chr_job_file_name = f'{WRKDIR}/plink/scripts/nabec.snp.list.chr{chrom}.jobs\n' 
        swarm_file.write(chr_job_file_name)



### run scripts

In [21]:
#for chrom in CHROMOSOMES:
!mkdir /data/LNG/anni/decon-eqtl/ppmi/plink/output/temp/V08


In [23]:
#format and make the swarm files executable
print('#run these commands at terminal:\n')
print(f'chmod +x {WRKDIR}/plink/scripts/*')
#for chrom in CHROMOSOMES:
SWARMFILENAME='{}/plink/scripts/run_ppmi_wb_eqtl.V08.swarm'.format(WRKDIR,visit)
#CHRFILENAME = '{}/plink/scripts/ppmi.snp.list.chr{}.jobs'.format(WRKDIR,chrom)
#LOGFILENAME='{}/plink/output/temp/run_ppmi_wb_eqtl.V08.log'.format(WRKDIR,chrom)
print(f'swarm -g 64 -t 24 -f run_ppmi_gene_snps.{visit}.swarm ')

#run these commands at terminal:

chmod +x /data/LNG/anni/decon-eqtl/ppmi/plink/scripts/*
swarm -g 64 -t 24 -f run_ppmi_gene_snps.V08.swarm 


In [4]:
## check number of gene files created matches earlier number
cmd = f'ls /data/LNG/anni/decon-eqtl/ppmi/plink/output/gene_snps/ | wc -l '
!{cmd}

26986


In [9]:
## check one
test = pd.read_csv(f'{WRKDIR}/plink/output/gene_snps/V08.ENSG00000238284.2.plink.snp.list', sep='\t')
test.head()

,GENEID,SNPID
0,ENSG00000238284.2,chr7:41661765:C:A
1,ENSG00000238284.2,chr7:41662670:G:A
2,ENSG00000238284.2,chr7:41662736:C:T
3,ENSG00000238284.2,chr7:41663058:G:A
4,ENSG00000238284.2,chr7:41663704:A:C


In [3]:
for x in CHROMOSOMES:
    print(f'cat ./temp/chr{x}/*.csv > ppmi.deconeqtl.results.chr{x}.cat.csv')

cat ./temp/chr1/*.csv > ppmi.deconeqtl.results.chr1.cat.csv
cat ./temp/chr2/*.csv > ppmi.deconeqtl.results.chr2.cat.csv
cat ./temp/chr3/*.csv > ppmi.deconeqtl.results.chr3.cat.csv
cat ./temp/chr4/*.csv > ppmi.deconeqtl.results.chr4.cat.csv
cat ./temp/chr5/*.csv > ppmi.deconeqtl.results.chr5.cat.csv
cat ./temp/chr6/*.csv > ppmi.deconeqtl.results.chr6.cat.csv
cat ./temp/chr7/*.csv > ppmi.deconeqtl.results.chr7.cat.csv
cat ./temp/chr8/*.csv > ppmi.deconeqtl.results.chr8.cat.csv
cat ./temp/chr9/*.csv > ppmi.deconeqtl.results.chr9.cat.csv
cat ./temp/chr10/*.csv > ppmi.deconeqtl.results.chr10.cat.csv
cat ./temp/chr11/*.csv > ppmi.deconeqtl.results.chr11.cat.csv
cat ./temp/chr12/*.csv > ppmi.deconeqtl.results.chr12.cat.csv
cat ./temp/chr13/*.csv > ppmi.deconeqtl.results.chr13.cat.csv
cat ./temp/chr14/*.csv > ppmi.deconeqtl.results.chr14.cat.csv
cat ./temp/chr15/*.csv > ppmi.deconeqtl.results.chr15.cat.csv
cat ./temp/chr16/*.csv > ppmi.deconeqtl.results.chr16.cat.csv
cat ./temp/chr17/*.csv > p

### delete all individual gene files

In [13]:
# cmd_rm_result = f'rm {WRKDIR}/plink/output/temp/{visit}/chr{chrom}/*'
# !{cmd_rm_result}

In [14]:
# ##checking output file
# snp_df = pd.read_csv(f'{WRKDIR}/plink/output/snp/V08.genes.chr22.plink.snp.list', sep='\t')
# print(snp_df.shape)
# snp_df.head()

### run command in terminal (within directory containing decon jar file)

In [ ]:
for chrom in CHROMOSOMES:
    cmd=f'mkdir {WRKDIR}/results/temp/chr{chrom}'
    #print(cmd)
    !{cmd}

In [42]:
for chrom in CHROMOSOMES:
    cmd=f'mkdir {WRKDIR}/scripts/chr{chrom}'
    #print(cmd)
    !{cmd}

In [ ]:
####job file per gene, swarm file per chrom

#read in the transcript annotations from txt
#gencode_annots_file = WRKDIR + '/gencode.v32.transcripts.txt.gz'
#gencode_annots_file = f'{WRKDIR}/gencode.v32.transcripts.txt.gz'
gencode = pd.read_csv(gencode_annots_file,sep='\t',comment='#', index_col=0)


## get gene list for eahc chrom again
#CHROMOSOMES = [22]
for chrom in CHROMOSOMES:
    chr_result_file_name = f'{WRKDIR}/results/{cohort}.deconeqtl.chr{chrom}.results.csv'
    with open(chr_result_file_name,'w') as chr_result_file: 
        header = '\tBasophils_pvalue\tEosinophils_pvalue\tLymphocytes_pvalue\tMonocytes_pvalue\tNeutrophils_pvalue\tBeta1_Basophils\tBeta2_Eosinophils\tBeta3_Lymphocytes\tBeta4_Monocytes\tBeta5_Neutrophils	Beta6_Basophils:GT\tBeta7_Eosinophils:GT\tBeta8_Lymphocytes:GT\tBeta9_Monocytes:GT\tBeta10_Neutrophils:GT'
        cmd_echo_header = f'{header} > {chr_result_file}\n'
        chr_result_file.write(cmd_echo_header)
        

    exp_df = pd.read_csv(f'{WRKDIR}/expression/{cohort}.aug2020.chr{chrom}.subset.pheno', sep='\t', index_col=0)


    #genome_index_file = f'{WRKDIR}/Homo_sapiens_assembly38.fasta.fai'
    chr_lengths_df = pd.read_csv(genome_index_file, header=None, sep='\t')

    max_dist = 1000000

    #for chrom in CHROMOSOMES:
    #chr_genes = gencode.loc[gencode['seqid'] == 'chr{}'.format(chrom)]
    chr_genes = gencode.loc[gencode['seqname'] == 'chr{}'.format(chrom)]

    chr_genes_id = set(chr_genes['gene_id']) & set(exp_df.index)
    print('{} transcripts detected on chromosome {}'.format(len(chr_genes_id),chrom))
    total_transcripts += int(len(chr_genes_id))
    chr_genes_df = chr_genes.loc[chr_genes['gene_id'].isin(chr_genes_id)]

    #compute the +/- cis proximal start and stop
    chr_genes_df['cisstart'] = chr_genes_df['start'] - max_dist
    chr_genes_df['cisstop'] = chr_genes_df['end'] + max_dist    

    #clean up if cis start or end is beyond chromosome beginning or end
    chr_genes_df.loc[chr_genes_df['cisstart'] < 0, 'cisstart'] = 0
    max_end = chr_lengths_df.loc[chr_lengths_df[0] == 'chr{}'.format(chrom),1].values[0]
    chr_genes_df.loc[chr_genes_df['cisstop'] > max_end, 'cisstop'] = max_end

    chr_genes_df = chr_genes_df[['gene_id', 'seqname', 'start', 'end', 'strand', \
                                            'cisstart', 'cisstop', 'gene_name']]
    
    ##for each gene write decon-eqtl command, add result to total chrom results file
    for gene_id in chr_genes_id:
        
        gene_job_file_name = f'{WRKDIR}/scripts/chr{chrom}/{cohort}.deconeqtl.{gene_id}.job' 
    
        with open(gene_job_file_name,'w') as gene_job_file: 
#             

            #gene_output_file = f'{WRKDIR}/output/V08.ppmi.deconResults.sept2020.chr{chrom}.csv'  

            gene_transcripts_df =  chr_genes_df.loc[chr_genes_df['gene_id'] == gene_id]

            #now format the biowulf job files
            command_java = f'java -Xmx100g \
                        -jar {WRKDIR}/Decon-eQTL-v*.*.*-jar-with-dependencies.jar \
                        -c {WRKDIR}/{fract_file} \
                        -e {WRKDIR}/expression/{cohort}.aug2020.chr{chrom}.subset.pheno\
                        -g {WRKDIR}/genotypes/{cohort}.aug2020.geno05maf05hwe000001.chr{chrom}.subset.traw \
                        --no_console \
                        -o {WRKDIR}/results/temp/chr{chrom} \
                        -of {cohort}.deconResults.{date}.{gene_id}.csv \
                        -sn {WRKDIR}/plink/output/gene_snps/chr{chrom}.{gene_id}.plink.snp.list\n \
                        > {WRKDIR}/logs/deconvolutionResults.{date}.{gene_id}.log\n'


            command_accum = f'awk '"'NR>1 {{ print }} '"' {}/results/temp/chr{}/{}.deconResults.{}.{}.csv >> {}\n'.format(WRKDIR,chrom,cohort,date,gene_id,chr_result_file_name)
            #command_rmv_snp = f'rm {WRKDIR}/output/gene_snps/V08.{gene}.chr{chrom}.plink.snp.list'
            command_rmv_result = f'rm {WRKDIR}/results/temp/chr{chrom}/{cohort}.deconResults.{date}.{gene_id}.csv'

            gene_job_file.write(command_java) 

            gene_job_file.write(command_accum)
            
    #format the swarm jobfile(s)
    ##collect all gene jobs file names to one file for swarm
    swarm_file_name = f'{WRKDIR}/scripts/run_{cohort}_deconeqtl_chr{chrom}.swarm'
    with open(swarm_file_name,'w') as swarm_file:
        for gene_id in chr_genes_id:
            gene_job_file_name = f'{WRKDIR}/scripts/chr{chrom}/{cohort}.deconeqtl.{gene_id}.job\n' 
            swarm_file.write(gene_job_file_name)


     
    

In [9]:
#format and make the swarm files executable
visit='V08'
## run swarm file for each chrom
print('#run these commands at terminal in Decon directory:\n')
#print(f'chmod +x {WRKDIR}/scripts/*')
for chrom in CHROMOSOMES:
    print(f'chmod +x {WRKDIR}/scripts/chr{chrom}/*')
    print(f'chmod +x {WRKDIR}/results/temp/chr{chrom}')
    #for visit in visits_oi:
    SWARMCHRFILENAME=f'{WRKDIR}/scripts/run_{cohort}_deconeqtl_chr{chrom}.swarm'
    #LOGFILENAME=f'{WRKDIR}/output/temp/ppmi.run.pairs.chr{chrom}.log'
    print(f'swarm -g 15 -t 4 -f {SWARMCHRFILENAME}')

#run these commands at terminal in Decon directory:

chmod +x /data/LNG/anni/decon-eqtl/ppmi/scripts/chr1/*
chmod +x /data/LNG/anni/decon-eqtl/ppmi/results/temp/chr1
swarm -g 15 -t 4 -f /data/LNG/anni/decon-eqtl/ppmi/scripts/run_ppmi_V08_deconeqtl_chr1.swarm
chmod +x /data/LNG/anni/decon-eqtl/ppmi/scripts/chr2/*
chmod +x /data/LNG/anni/decon-eqtl/ppmi/results/temp/chr2
swarm -g 15 -t 4 -f /data/LNG/anni/decon-eqtl/ppmi/scripts/run_ppmi_V08_deconeqtl_chr2.swarm
chmod +x /data/LNG/anni/decon-eqtl/ppmi/scripts/chr3/*
chmod +x /data/LNG/anni/decon-eqtl/ppmi/results/temp/chr3
swarm -g 15 -t 4 -f /data/LNG/anni/decon-eqtl/ppmi/scripts/run_ppmi_V08_deconeqtl_chr3.swarm
chmod +x /data/LNG/anni/decon-eqtl/ppmi/scripts/chr4/*
chmod +x /data/LNG/anni/decon-eqtl/ppmi/results/temp/chr4
swarm -g 15 -t 4 -f /data/LNG/anni/decon-eqtl/ppmi/scripts/run_ppmi_V08_deconeqtl_chr4.swarm
chmod +x /data/LNG/anni/decon-eqtl/ppmi/scripts/chr5/*
chmod +x /data/LNG/anni/decon-eqtl/ppmi/results/temp/chr5
swarm -g 1

In [ ]:
## all results will be per chrom
## need to load each chrom, merge

result_df = pd.read_csv(f'{WRKDIR}/results/{cohort}.deconeqtl.chr1.results.csv', sep = '\t')
print('chrom1: '+str(result_df.shape))

def load_result(chrom, return_df):
    next_df = pd.read_csv(f'{WRKDIR}/results/{cohort}.deconeqtl.chr{chrom}.results.csv', sep = '\t')
    result_df = result_df.merge(next_df)
    print(f'chrom{chrom}: '+str(result_df.shape))
    return result_df

CHROMOSOMES_sub=[str(x) for x in list(range(2,23))]
for chrom in CHROMOSOMES_sub:
    result_df = load_result(chrom, result_df)
    
result_df.head()